# Formulation of Maxwell's Equations in Cartesian Coordinates and Flat Spacetime - An Overview

## Author: Terrence Pierre Jacques

## This tutorial notebook outlines two formulations of Maxwell's equations.


### This module serves as a stepping stone towards solving Einstein's equations using NRPy+. As such, the [tutorial notebook on the scalar wave equation in curvilinear coordinates](Tutorial-ScalarWaveCurvilinear.ipynb) is *required* reading before beginning this module. That module, as well as its own prerequisite [module on reference metrics within NRPy+](Tutorial-Reference_Metric.ipynb) provides the needed overview of how NRPy+ handles reference metrics.

# Introduction 

To numerically solve Einstein's equations, a system of coupled, non-linear equations, the spatial coordinates may be decoupled from the time coordinate. In doing so, numerical solutions may be marched forward in time, using the [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) ([NRPy+ tutorial here](Tutorial-Method_of_Lines-C_Code_Generation.ipynb)). This is done using the 3 + 1 decomposition, originally done using the ADM formalism. However, it was found that using this formulation of Einstein's equations resulted in unstable simulations, due to its weakly hyperbolic system of equations. Hyperbolicity is the notion of the wave-like behavior of the numerical solution for a given system of PDE's. The ADM formulation of Einstein's equations involved mixed second derivatives, which give rise to a weakly hyperbolic system.

In contrast, the BSSN formalism removes these mixed second derivatives, resulting in a strongly hyperbolic system of equations. In this module, we follow the work of [Knapp, Walker & Baumgarte (2002)](https://arxiv.org/abs/gr-qc/0201051), showcasing the difference between using strongly versus weakly hyperbolic systems, when solving a physical problem with Maxwell's equations in vacuum.
 
 ### A Note on Notation:

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.


<a id='top'></a>
$$\label{top}$$

# Table of Contents:  

1. [Step 1](#step1): Maxwell's Equations in Vacuum - System I (ADM-like)
1. [Step 2](#step2): Maxwell's Equations in Vacuum - System II (BSSN-like)
1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='step1'></a>

# Step 1: Maxwell's Equations in Vacuum - System I (ADM-like) \[Back to [top](#top)\]
$$\label{step1}$$

We begin with Maxwell's equations in vacuum, i.e. no source terms, in flat space and in [Gaussian](https://en.wikipedia.org/wiki/Gaussian_units) and $c = 1$ units, given by 

\begin{align}
&\vec{\nabla} \cdot \vec{E} = 0, \\
&\vec{\nabla} \cdot \vec{B} = 0, \\
&\frac{\partial \vec{B}}{\partial t} = -\vec{\nabla} \times \vec{E}, \\
&\frac{\partial \vec{E}}{\partial t} = \vec{\nabla} \times \vec{B},
\end{align}

and we have the associated potentials,

\begin{align}
\frac{\partial \vec{A}}{\partial t} &= -\vec{E} -\vec{\nabla}\varphi , \\
\vec{B} &= \vec{\nabla} \times \vec{A}.
\end{align}

Now, replacing $\vec{B}$ with $\vec{\nabla} \times \vec{A}$ in Ampere's law, and using the standard identity

$$
\vec{\nabla} \times \left( \vec{\nabla} \times \vec{A} \right) = \vec{\nabla} \left( \vec{\nabla} \cdot \vec{A} \right) - \nabla^2 \vec{A},
$$

we may write

$$
\vec{\nabla} \times \vec{B} = \vec{\nabla} \times \left( \vec{\nabla} \times \vec{A} \right) = \vec{\nabla} \left( \vec{\nabla} \cdot \vec{A} \right) - \nabla^2 \vec{A}.
$$

Thus, our time evolution equations become

\begin{align}
\frac{\partial \vec{A}}{\partial t} &= -\vec{E} -\vec{\nabla}\varphi , \\
\frac{\partial \vec{E}}{\partial t} &= \vec{\nabla} \left( \vec{\nabla} \cdot \vec{A} \right) - \nabla^2 \vec{A}.
\end{align}


Using index notation, in Cartesian coordinates we have 

\begin{align}
\partial_t A^i &= -E^i - \partial^i \varphi, \\
\partial_t E^i &= \partial^i \partial_j A^j - \partial_j \partial^j A^i.
\end{align}

Note the presence of the mixed second derivative above, and that in our system of equations we have 6 equations (since $\vec{A}$ and $\vec{E}$ each have 3 components), but 7 unknowns (components of $\vec{A}$ and $\vec{E}$, and the scalar potential $\varphi$). Thus, we’ll add a time evolution equation to $\varphi$ as well, which amounts to choosing a gauge. In particular we’ll choose the [Lorenz gauge](https://en.wikipedia.org/wiki/Lorenz_gauge_condition):

$$
\partial_t \varphi = -\vec{\nabla} \cdot \vec{A},
$$

and using index notation, in Cartesian coordinates,

$$
\partial_t \varphi = -\partial_i A^i.
$$

Furthermore, note that because we are working in vacuum, in Cartesian coordinates we have the constraints

\begin{align}
\partial_i E^i &= 0, \\
\partial_i B^i &= 0,
\end{align}

But since $\vec{B} = \vec{\nabla} \times \vec{A}$, the divergence of $\vec{B}$ is automatically satisfied (excercise for the reader).

The right hand sides (RHSs) of our evolution equations are thus

\begin{align}
\partial_t A^i &= -E^i - \partial^i \varphi, \\
\partial_t E^i &= \partial^i \partial_j A^j - \partial_j \partial^j A^i, \\
\partial_t \varphi &= -\partial_i A^i,
\end{align}

subject to the constraint

$$
\mathcal{C} \equiv \partial_i E^i = 0.
$$

Tracking the departure of our numerical results from this constraint helps us keep track of the numerical error. We refer to this system of equations as System I.

In [this tutorial](Tutorial-VacuumMaxwell_Cartesian_RHSs.ipynb) the RHSs and constraint equations are implemented in NRPy+. The system is then evolved in time within a [start-to-finish notebook](Tutorial-Start_to_Finish-Solving_Maxwells_Equations_in_Vacuum-Cartesian.ipynb), using initial data defined in [this tutorial](Tutorial-VacuumMaxwell_InitialData.ipynb), showcasing the **instability** of the system as a result of the presence of the mixed second derivative for $E^i$.

<a id='step2'></a>

# Step 2: Maxwell's Equations in Vacuum - System II (BSSN - like) \[Back to [top](#top)\]
$$\label{step2}$$

As dicussed in the introduction and following the previous section, to maintain numerical stability and accuracy we remove the mixed 2nd derivative $\partial^i \partial_j A^j$ in the equation for $E^i$ by introducing

\begin{align}
\Gamma &\equiv \partial_i A^i, \\
\partial_t \Gamma &= \partial_i \partial_t A^i = -\partial_i E^i - \partial_i \partial^i \varphi \\
 &= - \partial_i \partial^i \varphi.
\end{align}

Thus, our evolution equations are 

\begin{align}
\partial_t A^i &= -E^i - \partial^i \varphi, \\
\partial_t E^i &= \partial^i \Gamma - \partial_j \partial^j A^i, \\
\partial_t \Gamma &= - \partial_i \partial^i \varphi, \\
\partial_t \varphi &= -\Gamma,
\end{align}

subject to the constraints

\begin{align}
\mathcal{G} &\equiv \Gamma - \partial_i A^i &= 0, \\
\mathcal{C} &\equiv \partial_i E^i &= 0.
\end{align}

We refer to this system of equations as System II.

In [this tutorial](Tutorial-VacuumMaxwell_Cartesian_RHSs.ipynb) the RHSs and constraints are implemented in NRPy+. The system is then evolved in time within a [start-to-finish notebook](Tutorial-Start_to_Finish-Solving_Maxwells_Equations_in_Vacuum-Cartesian.ipynb), using initial data defined in [this tutorial](Tutorial-VacuumMaxwell_InitialData.ipynb), showcasing the **stability** of the system as a result of the absence of the mixed second derivative for $E^i$.

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#top)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-VacuumMaxwell_formulation_Cartesian.pdf](Tutorial-VacuumMaxwell_formulation_Cartesian.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [1]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-VacuumMaxwell_formulation_Cartesian")

Created Tutorial-VacuumMaxwell_formulation_Cartesian.tex, and compiled
    LaTeX file to PDF file Tutorial-VacuumMaxwell_formulation_Cartesian.pdf
